In [776]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import json
import seaborn as sns
from matplotlib.pyplot import figure
import datetime
import plotly.express as px
from sklearn.cluster import KMeans
import plotly.graph_objs as go
import numpy as np

In [117]:
def to_time(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x >=0)and(x <= 4):
        return'Late Night'

In [625]:
df=pd.read_csv("USvideos.csv")

In [733]:
df2=pd.read_csv("USvideos.csv")

In [626]:
with open('US_category_id_2.json', 'r') as f:
    data = json.load(f)

In [627]:
cat_id={}
for i in data['items']:
    cat_id[i['id']]=i['snippet']['title']

In [628]:
category_id_df=pd.DataFrame(data=cat_id.items(),columns=["category_id","category"])

In [629]:
category_id_df['category_id']=category_id_df['category_id'].astype("int")

In [577]:
df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125785,iyHG7Jvh2Ik,Janet & Bonnie Breakdown! | Update Info!,2022-04-23T15:09:12Z,UCmG2EhfOwSjpPMX4LjGY__A,KairosTime Gaming,20,2022-04-28T00:00:00Z,Brawl Stars|BrawlStars|Kairos|KairosTime|Kairo...,321949,20331,0,2069,https://i.ytimg.com/vi/iyHG7Jvh2Ik/default.jpg,False,False,Sponsored by Mobile Legends Bang Bang! Downloa...
125786,KnpugtdWfVw,Opposite Twins Make-Over Eachother for Coachel...,2022-04-22T23:11:29Z,UCuVHOs0H5hvAHGr8O4yIBNQ,Niki and Gabi,26,2022-04-28T00:00:00Z,coachella|coachella 2022|opposite twins make o...,476256,24139,0,508,https://i.ytimg.com/vi/KnpugtdWfVw/default.jpg,False,False,Opposite Twins Make-Over Each other for Coache...
125787,o_t1QlmHckw,Yahritza Y Su Esencia - Dejalo ir,2022-04-22T04:00:57Z,UC7ptwo_0Npl5Bji31j868fA,Yahritza Y Su Esencia,10,2022-04-28T00:00:00Z,Yahritza|Yahritza Martinez|Yahritza Y Su Esenc...,1263504,67713,0,2052,https://i.ytimg.com/vi/o_t1QlmHckw/default.jpg,False,False,Yahritza Y Su Esencia - Dejalo ir Video Letra ...
125788,LTpKec0L4Bc,"Southside, Future - Hold That Heat (Official A...",2022-04-22T04:00:10Z,UCoKENLYd3bL8pSU8XWGh3ig,SouthsideVEVO,10,2022-04-28T00:00:00Z,new music|epic|epic records|look in my eyes|al...,538114,11586,0,433,https://i.ytimg.com/vi/LTpKec0L4Bc/default.jpg,False,False,Southside & Future feat. Travis Scott - Hold T...


In [630]:
df=df[df['likes']!=0]

In [631]:
df.reset_index(drop=True,inplace=True)

In [632]:
vid_id=list(set(df['video_id'].values))
max_view=0
df_=pd.DataFrame()
for vid in vid_id:
    views=df[df['video_id']==vid]['views'].values
    max_view=max(views) if len(views)>1 else views[0]
    row=df[(df['video_id']==vid)&(df['views']==max_view)]
    df_=pd.concat([df_,row])

In [635]:
df=df_

In [636]:
df.reset_index(drop=True,inplace=True)
df.drop_duplicates(subset=['video_id'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [638]:
df=df.merge(category_id_df,on='category_id',how='inner')

In [639]:
df

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
0,UelDrZ1aFeY,17.22.12,The Beatles - Something,TheBeatlesVEVO,10,2017-12-15T16:14:30.000Z,"something the beatles video|""something the bea...",572627,24313,226,1899,https://i.ytimg.com/vi/UelDrZ1aFeY/default.jpg,False,False,False,Official site: http://www.thebeatles.com\nFace...,Music
1,YbHdBoHMxK4,17.22.11,Parson James - Only You (Lyric Video),ParsonJamesVEVO,10,2017-11-17T08:00:02.000Z,"Only You|""Parson James""|""Pop""|""RCA Records Label""",35511,2016,11,69,https://i.ytimg.com/vi/YbHdBoHMxK4/default.jpg,False,False,False,“Only You” by Parson James Available Now!\n\ni...,Music
2,BspHjvU11y4,18.09.06,Kelly Clarkson - Meaning of Life [Official Video],Kelly Clarkson,10,2018-05-13T13:00:41.000Z,"Kelly Clarkson|""Meaning of Life""|""mol""|""Offici...",2421584,49501,1163,2507,https://i.ytimg.com/vi/BspHjvU11y4/default.jpg,False,False,False,MEANING OF LIFE available now: https://Atlanti...,Music
3,aDBMVKtcG2Q,18.25.04,It was... bonkers. Tom Hiddleston on life as L...,BBC Radio 1,10,2018-04-20T15:27:10.000Z,"loki|""avengers""|""infinity war""|""thor""|""tom hid...",405688,15661,122,1465,https://i.ytimg.com/vi/aDBMVKtcG2Q/default.jpg,False,False,False,Watch Becoming... Loki in full on BBC iPlayer ...,Music
4,GxLYCKB8v2w,17.07.12,First Aid Kit - Fireworks (Official Lyric Video),First Aid Kit,10,2017-12-01T05:00:00.000Z,"First Aid Kit|""Fireworks""|""Ruins""|""First Aid K...",21513,770,4,63,https://i.ytimg.com/vi/GxLYCKB8v2w/default.jpg,False,False,False,Fireworks off First Aid Kit's upcoming album '...,Music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,eIns8Y0xegE,18.09.02,"Unrigged Live! with Jennifer Lawrence, Tig Not...",Represent.Us,29,2018-02-04T06:12:37.000Z,"Unrig|""Unrig the System""|""Unrig the System Sum...",11736,114,34,30,https://i.ytimg.com/vi/eIns8Y0xegE/default.jpg,False,False,False,We're live at Unrig the System Summit in New O...,Nonprofits & Activism
6321,V6ElE2xs48c,18.06.06,Game of Zones - S5:E5: The Isle of Van Gundy,Bleacher Report,43,2018-05-10T21:01:22.000Z,"bleacher report|""br""|""nba""|""Stan Van Gundy""|""J...",1351321,22587,616,1649,https://i.ytimg.com/vi/V6ElE2xs48c/default.jpg,False,False,False,"On an island full of Van Gundys, Stan gets tal...",Shows
6322,Q1CFfU2gXHw,18.30.01,Apple HomePod: Everything to know before you b...,CNET,43,2018-01-24T20:08:16.000Z,"Apple|""Homepod""|""smart home""|""Google Home""|""Am...",72607,1130,113,264,https://i.ytimg.com/vi/Q1CFfU2gXHw/default.jpg,False,False,False,We'll break down all the new Apple HomePod det...,Shows
6323,7_FJUSBFbJM,18.29.04,Game of Zones - Game of Zones - S5:E1: 'A Gold...,Bleacher Report,43,2018-04-12T21:00:12.000Z,"bleacher report|""br""|""nba""|""game of zones""|""go...",1445949,33261,475,2174,https://i.ytimg.com/vi/7_FJUSBFbJM/default.jpg,False,False,False,"In the season 5 premiere of Game of Zones, the...",Shows


In [445]:
df_.drop(['category_x','category_y'],axis=1,inplace=True)

In [640]:
df['publish_time']=df['publish_time'].apply(lambda x:pd.to_datetime(x))

df=df.sort_values(by='publish_time',ascending=False)

df.reset_index(drop=True,inplace=True)

df['publish_time_day']=df['publish_time'].dt.hour

df['publish_time_day']=df['publish_time_day'].apply(lambda x:to_time(x))



weekday_map={0:"monday",1:"tuesday",2:"wednesday",3:"thursday",4:"friday",5:"saturday",6:"sunday"}
df['publishedAt_day']=df['publish_time'].apply(lambda x:weekday_map[datetime.datetime.weekday(x)])

In [647]:
df['publishedAt_day_time']=df['publishedAt_day'].astype(str)+" "+df["publish_time_day"].astype(str)

In [648]:
df['likes_dislikes_ratio']=df['likes']/(df['dislikes']+1)

In [649]:
df.drop_duplicates(subset=['video_id'],inplace=True)

In [728]:
df['response_type']=df['likes_dislikes_ratio'].apply(lambda x:"good response"if x>=2 else "bad response")

In [819]:
df['publish_year']=df['publish_time'].dt.year
df['publish_hour']=df['publish_time'].dt.hour

In [708]:
fig=px.histogram(df, x="category")
fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

In [828]:
fig=px.histogram(df, x="publish_hour",color='response_type')
fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

In [707]:
fig = px.box(df, x="category", y="views",width=1000, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.add_annotation(
        x=0.5,
        y=200e6,
        xref="x",
        yref="y",
        text="Music",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.add_annotation(
        x=1.5,
        y=155e6,
        xref="x",
        yref="y",
        text="Entertainment",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.update_layout(xaxis={'categoryorder':'total descending'}) # put boxplots in descending order based on values

fig.show()

In [718]:
fig = px.box(df, x="category", y="comment_count",width=1000, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout(xaxis={'categoryorder':'total descending'}) # put boxplots in descending order based on values

fig.show()

In [732]:

fig = px.box(df, x="category", y="likes",width=1000, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.add_annotation(
        x=0.5,
        y=5.5e6,
        xref="x",
        yref="y",
        text="Music",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.add_annotation(
        x=1.5,
        y=3.1e6,
        xref="x",
        yref="y",
        text="Entertainment",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.update_layout(xaxis={'categoryorder':'total descending'}) # put boxplots in descending order based on values

fig.show()

In [731]:
fig = px.box(df, x="category", y="likes_dislikes_ratio",width=1000, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout(xaxis={'categoryorder':'total descending'}) # put boxplots in descending order based on values

fig.show()

In [730]:
fig=px.histogram(df[df['category']=="Music"], x="channel_title",color='response_type')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [723]:
df[(df['channel_title']=="TheEllenShow")&(df['likes_dislikes_ratio']<3)]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,category,publish_time_day,publishedAt_day,publishedAt_day_time,likes_dislikes_ratio,publish_year
2028,Ob48twC1yf8,18.28.02,Parkland Shooting Survivors on the 'March for ...,TheEllenShow,24,2018-02-23 14:01:11+00:00,"Ellen|""degeneres""|""ellen degeneres""|""the ellen...",1159873,28883,10433,...,True,False,False,"Emma Gonzalez, Cameron Kasky, and Jaclyn Corin...",Entertainment,Noon,friday,friday Noon,2.768162,2018


In [729]:
fig=px.histogram(df[df['category']=="Entertainment"], x="channel_title",color='response_type')
fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

In [713]:

fig = px.box(df, x="category", y="views", color="publishedAt_day",width=1000, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout(xaxis={'categoryorder':'total descending'}) # put boxplots in descending order based on values

fig.show()

In [825]:
df

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,video_error_or_removed,description,category,publish_time_day,publishedAt_day,publishedAt_day_time,likes_dislikes_ratio,publish_year,response_type,publish_hour
0,8O-ZUXGcuWU,18.14.06,Nintendo @ E3 2018: Day 2,Nintendo,20,2018-06-14 01:31:53+00:00,"nintendo|""play""|""play nintendo""|""game""|""gamepl...",1549110,21905,1516,...,False,ESRB Rating: RATING PENDING to TEEN. Rating Pe...,Gaming,Late Night,thursday,thursday Late Night,14.439684,2018,good response,1
1,Gi56dSh8Fq8,18.14.06,Gourmet Chef Makes A Big Mac Super Fancy,BuzzFeedVideo,22,2018-06-13 18:00:32+00:00,"mcdonalds|""big mac""|""fancy""|""fast food""|""jacqu...",402418,10070,3303,...,False,"It's your good ol' McDonald's Big Mac, but lik...",People & Blogs,Eve,wednesday,wednesday Eve,3.047821,2018,good response,18
2,-9rdDeWzvsU,18.14.06,Stampede - Alexander Jean Ft. Lindsey Stirling,Lindsey Stirling,10,2018-06-13 16:00:24+00:00,"lindsey|""lindsay""|""violin""|""dubstep""|""electron...",296615,38671,463,...,False,Lindsey Stirling & Evanescence Co-Headline Sum...,Music,Noon,wednesday,wednesday Noon,83.342672,2018,good response,16
3,MrMAfMbZOmg,18.14.06,Troye Sivan - Dance To This (Official Audio) f...,TroyeSivanVEVO,10,2018-06-13 16:00:01+00:00,"Troye|""Sivan""|""Dance""|""To""|""This""|""EMI""|""Recor...",1473589,214511,2011,...,False,“Dance to This” feat. Ariana Grande available ...,Music,Noon,wednesday,wednesday Noon,106.615805,2018,good response,16
4,JGm9Y_hFqNk,18.14.06,First Take reacts: Kyrie Irving says contract ...,ESPN,17,2018-06-13 15:06:08+00:00,"espn|""espn live""|""boston celtics""|""kyrie irvin...",812832,7701,320,...,False,First Take's Stephen A. Smith and Max Kellerma...,Sports,Noon,wednesday,wednesday Noon,23.990654,2018,good response,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,rO_mAQC9bv0,17.20.11,Amazon.com commercial Xmas 1999Magnetic Whale Art,astroboy1960,1,2009-03-10 00:27:20+00:00,"amazon|""commerical""|""y2k""|""sing""|""along""|""mitc...",15476,34,1,...,False,IMHO the best of Amazon's Christmas 1999 ads. ...,Film & Animation,Late Night,tuesday,tuesday Late Night,17.000000,2009,good response,0
6321,Y8JGfi4MJ8k,17.28.11,SAOIRSE RONAN - MORONIC (IRONIC),flirppy242,10,2008-08-07 12:16:58+00:00,"SAOIRSE|""RONAN""|""MORONIC""|""IRONIC""|""IZZIE""|""CO...",26830,79,1,...,False,EL NOVIO DE MI MADRE\r\nI COULD NEVER BE A WOMAN,Music,Morning,thursday,thursday Morning,39.500000,2008,good response,12
6322,wFEAoF7RC7Y,17.28.11,Behind The Sounds: That's Not Me,BehindTheSounds,10,2008-06-17 00:07:56+00:00,"Beach|""Boys""|""Brian""|""Wilson""|""Dennis""|""Carl""|...",86897,469,4,...,False,A video detailing the creative process behind ...,Music,Late Night,tuesday,tuesday Late Night,93.800000,2008,good response,0
6323,UQtt9I6c-YM,18.09.01,Kramer vs Kramer-Clou Scene,Livia Giustiniani,1,2008-04-05 18:22:40+00:00,"Meryl|""Streep""|""kramer""|""vs""|""dustin""|""hoffman...",50168,46,6,...,False,Poor Meryl...she was really scaredxD,Film & Animation,Eve,saturday,saturday Eve,6.571429,2008,good response,18


In [603]:
avg_music_views=music_views.sort_values(by="view_count",ascending=False)

In [816]:
df

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,ratings_disabled,video_error_or_removed,description,category,publish_time_day,publishedAt_day,publishedAt_day_time,likes_dislikes_ratio,publish_year,response_type
0,8O-ZUXGcuWU,18.14.06,Nintendo @ E3 2018: Day 2,Nintendo,20,2018-06-14 01:31:53+00:00,"nintendo|""play""|""play nintendo""|""game""|""gamepl...",1549110,21905,1516,...,False,False,ESRB Rating: RATING PENDING to TEEN. Rating Pe...,Gaming,Late Night,thursday,thursday Late Night,14.439684,2018,good response
1,Gi56dSh8Fq8,18.14.06,Gourmet Chef Makes A Big Mac Super Fancy,BuzzFeedVideo,22,2018-06-13 18:00:32+00:00,"mcdonalds|""big mac""|""fancy""|""fast food""|""jacqu...",402418,10070,3303,...,False,False,"It's your good ol' McDonald's Big Mac, but lik...",People & Blogs,Eve,wednesday,wednesday Eve,3.047821,2018,good response
2,-9rdDeWzvsU,18.14.06,Stampede - Alexander Jean Ft. Lindsey Stirling,Lindsey Stirling,10,2018-06-13 16:00:24+00:00,"lindsey|""lindsay""|""violin""|""dubstep""|""electron...",296615,38671,463,...,False,False,Lindsey Stirling & Evanescence Co-Headline Sum...,Music,Noon,wednesday,wednesday Noon,83.342672,2018,good response
3,MrMAfMbZOmg,18.14.06,Troye Sivan - Dance To This (Official Audio) f...,TroyeSivanVEVO,10,2018-06-13 16:00:01+00:00,"Troye|""Sivan""|""Dance""|""To""|""This""|""EMI""|""Recor...",1473589,214511,2011,...,False,False,“Dance to This” feat. Ariana Grande available ...,Music,Noon,wednesday,wednesday Noon,106.615805,2018,good response
4,JGm9Y_hFqNk,18.14.06,First Take reacts: Kyrie Irving says contract ...,ESPN,17,2018-06-13 15:06:08+00:00,"espn|""espn live""|""boston celtics""|""kyrie irvin...",812832,7701,320,...,False,False,First Take's Stephen A. Smith and Max Kellerma...,Sports,Noon,wednesday,wednesday Noon,23.990654,2018,good response
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,rO_mAQC9bv0,17.20.11,Amazon.com commercial Xmas 1999Magnetic Whale Art,astroboy1960,1,2009-03-10 00:27:20+00:00,"amazon|""commerical""|""y2k""|""sing""|""along""|""mitc...",15476,34,1,...,False,False,IMHO the best of Amazon's Christmas 1999 ads. ...,Film & Animation,Late Night,tuesday,tuesday Late Night,17.000000,2009,good response
6321,Y8JGfi4MJ8k,17.28.11,SAOIRSE RONAN - MORONIC (IRONIC),flirppy242,10,2008-08-07 12:16:58+00:00,"SAOIRSE|""RONAN""|""MORONIC""|""IRONIC""|""IZZIE""|""CO...",26830,79,1,...,False,False,EL NOVIO DE MI MADRE\r\nI COULD NEVER BE A WOMAN,Music,Morning,thursday,thursday Morning,39.500000,2008,good response
6322,wFEAoF7RC7Y,17.28.11,Behind The Sounds: That's Not Me,BehindTheSounds,10,2008-06-17 00:07:56+00:00,"Beach|""Boys""|""Brian""|""Wilson""|""Dennis""|""Carl""|...",86897,469,4,...,False,False,A video detailing the creative process behind ...,Music,Late Night,tuesday,tuesday Late Night,93.800000,2008,good response
6323,UQtt9I6c-YM,18.09.01,Kramer vs Kramer-Clou Scene,Livia Giustiniani,1,2008-04-05 18:22:40+00:00,"Meryl|""Streep""|""kramer""|""vs""|""dustin""|""hoffman...",50168,46,6,...,False,False,Poor Meryl...she was really scaredxD,Film & Animation,Eve,saturday,saturday Eve,6.571429,2008,good response
